In [1]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm
import seaborn as sns
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
from underwriting import msrp_band_analyzer

%load_ext autoreload
%autoreload 2

/Users/victoria/GitHub/venvs/bespoke/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
COMPANY_IDENTIFIER_LIST = [
    'DL',
    'DW',
    'EMA',
    'EMM',
    'EMT',
    'EMF',
    'ST',
    'GRG',
    'EL',
    'VS',
]

In [3]:
mba = msrp_band_analyzer.MSRPBand()

In [4]:
mba.update_company_data(COMPANY_IDENTIFIER_LIST)


Verifying download summaries for license MR282376...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11
Found bad download summary for license MR282376 on date 2022-03-31
Found bad download summary for license MR282376 on date 2022-03-24

Verifying download summaries for license MR283369...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11
Found bad download summary for license MR283369 on date 2022-03-05

Verifying download summaries for license 402-00840...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summaries for license 403-01293...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summaries for license 402R-00545...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summaries for license 403R-01201...
Earliest download summary: 2020-01-01
Latest download summary: 2022-04-11

Verifying download summar

/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['count_measure_from_product_name'][count_measure_non_na_index] = df['count_measure_from_product_name'][count_measure_non_na_index].apply(mba_util.extract_count_units)
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['extracted_units'][idx] = df[measure_column][idx].apply(mba_util.EXTRACTED_MEASUREMENT_COLUMNS[measure_column])
/Users/victoria/dev/platform/services/api-server/script

### Combining same product categories with different measurements ### 	


/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[measure_ratio_column_name][includes_measurement] = self.unit_conversion_ratio(measurement, measurement_unit)
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[measure_ratio_column_name][includes_measurement] = self.unit_conversion_ratio(measurement, measurement_unit)
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:323: SettingWithCopyWarn

### Breaking down product names into different brands ### 	


/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:357: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  (df[product_name].str.contains("^(1 ml)") == False) &
/Users/victoria/dev/platform/services/api-server/scripts/underwriting/msrp_band_analyzer.py:357: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  (df[product_name].str.contains("^(1 ml)") == False) &


### Available combined product category name in costs dataframe ### 	
['concentrate' 'infused' 'vape cartridge' 'edible' 'pre-roll infused'
 'shake/trim' 'flower' 'other concentrate' 'buds' 'pre-roll flower'
 'extract' 'pre-roll leaf' 'tincture' 'topical' 'capsule' 'shake'
 'wet whole plants' 'vape product' 'raw pre-rolls' 'seeds'
 'infused butter/oil' 'infused pre-rolls' 'kief' 'infused beverage'
 'shake/trim allocated for extraction' 'clone - cutting']
### Available combined product category name in sales dataframe ### 	
['other concentrate' 'edible' 'pre-roll flower' 'flower' 'buds'
 'concentrate' 'infused' 'shake/trim' 'extract' 'vape cartridge'
 'pre-roll infused' 'pre-roll leaf' 'topical' 'immature plants' 'capsule'
 'tincture' 'shake' 'raw pre-rolls' 'vape product'
 'shake/trim allocated for extraction' 'infused pre-rolls'
 'infused butter/oil' 'seeds' 'infused beverage' 'kief' 'clone - cutting']


In [5]:
###
df_sales = mba.company_sales_df.copy()

In [6]:
df_sales.head()

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,tx_price_per_unit,date_in_month,bad_numbers_from_product_name,extracted_units,letter_gram_measure_from_product_name,count_measure_from_product_name,gram_measure_from_product_name,oz_measure_from_product_name,adjusted_tx_price_per_unit,measurement_ratio_vs_gram,combined_product_category,brand_breakable_by_dash_boolean,brands,brands_by_category
2356560,C10-0000169-LIC,0005181809,inactive,Consumer,2020-01-02 08:16:34+00:00,2020-01,2,33.76,inactive,1782428,1A4060300004D62000000186,VVS - Vape - 0.3g - Rose Gold - Indica - XXX OG,Other Concentrate (weight - each),Each,1.0,16.88,16.880000,2020-01,NaN,0.3,0.3g,NaN,NaN,NaN,56.266667,0.3,other concentrate,True,VVS,VVS (Other Concentrate (weight - each))
2336473,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,1804087,1A4060300004B03000009761,LOL Edibles - Edible - 100mg - Doob Cube - Ras...,Edible (weight - each),Each,1.0,1.00,1.000000,2020-01,NaN,0.1,100mg,NaN,NaN,NaN,10.000000,0.1,edible,True,LOL Edibles,LOL Edibles (Edible (weight - each))
2336474,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,1782339,1A4060300004D62000000057,Puffy Delivery - Pre Roll - 0.8g - Sativa - Fa...,Pre-Roll Flower,Each,1.0,8.68,8.680000,2020-01,NaN,0.8,0.8g,NaN,NaN,NaN,10.850000,0.8,pre-roll flower,True,Puffy Delivery,Puffy Delivery (Pre-Roll Flower)
2336475,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,2111948,1A4060300004B03000010682,THC Design - Flower - 3.5g - Sativa - Crescendo,Flower,Grams,3.5,1.00,0.285714,2020-01,NaN,NaN,3.5g,NaN,NaN,NaN,0.285714,1.0,flower,True,THC Design,THC Design (Flower)
2336476,C10-0000169-LIC,0007521891,inactive,Consumer,2020-01-22 13:41:39+00:00,2020-01,5,36.58,inactive,1803899,1A4060300004B03000009717,LOL Edibles - Edible - 100mg - Doob Cube - Blu...,Edible (weight - each),Each,1.0,17.22,17.220000,2020-01,NaN,0.1,100mg,NaN,NaN,NaN,172.200000,0.1,edible,True,LOL Edibles,LOL Edibles (Edible (weight - each))


In [ ]:
###
df_sales.groupby(['tx_product_category_name','tx_unit_of_measure']).count().sort_values(by = 'tx_unit_of_measure',ascending = False)

# 1. uom - each:
- Using same UOM (ex. gram): comes in different quantity
- Using different UOM (ex. blend of gram and oz)

In [ ]:
## examples

In [ ]:
df_sales[(df_sales['tx_product_category_name'] == 'Other Concentrate (weight - each)')&(df_sales['tx_product_name'] == 'Wave - Cartridge - 0.5g - Hybrid - Lemon')]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()





In [ ]:
df_sales[(df_sales['tx_product_category_name'] == 'Other Concentrate (weight - each)')&(df_sales['tx_product_name'] == 'Thclear - 1g - Honey Pot - Skywalker OG')]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()





In [ ]:
## examples

In [ ]:

df_sales[(df_sales['tx_product_category_name'] == 'Edible (volume - each)')&(df_sales['tx_product_name'] == "Uncle Arnie's Iced Tea Lemonade Beverage 8oz Glass")]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()

In [ ]:

df_sales[(df_sales['tx_product_category_name'] == 'Edible (volume - each)')&(df_sales['tx_product_name'] == "Hapy - Edible - 100mg - Syrup Shot - Tropical Mix")]\
[['tx_product_category_name','tx_product_name','tx_unit_of_measure','tx_quantity_sold']].head()

# 2. how do we do regex

## 1. remove bad numbers

In [ ]:
df_sales[df_sales['bad_numbers_from_product_name'].notna()][['original_product_name','bad_numbers_from_product_name','tx_product_name']].head()

In [ ]:
df_sales[df_sales['bad_numbers_from_product_name'].notna()].shape[0] / df_sales.shape[0]

## 2. grab letter grams / gram / oz

In [ ]:
df_sales[df_sales['letter_gram_measure_from_product_name'].notna()][['tx_product_name','letter_gram_measure_from_product_name',]].head()




In [ ]:
df_sales[df_sales['letter_gram_measure_from_product_name'].notna()].shape[0] / df_sales.shape[0]

In [ ]:

df_sales[(df_sales['gram_measure_from_product_name'].notna()) & (df_sales['gram_measure_from_product_name'] == 'gram')][['tx_product_name','gram_measure_from_product_name']]\
.head()




In [ ]:
df_sales[df_sales['gram_measure_from_product_name'].notna()].shape[0] / df_sales.shape[0]

In [ ]:


df_sales[(df_sales['oz_measure_from_product_name'].notna())][['tx_product_name','oz_measure_from_product_name']]\
.head()




In [ ]:
df_sales[df_sales['oz_measure_from_product_name'].notna()].shape[0] / df_sales.shape[0]

# 3. combine 

In [ ]:
df_sales[df_sales['letter_gram_measure_from_product_name'].notna()][['tx_product_name','letter_gram_measure_from_product_name','extracted_units']].head()




In [ ]:

df_sales[(df_sales['gram_measure_from_product_name'].notna()) & (df_sales['gram_measure_from_product_name'] == 'gram')][['tx_product_name','gram_measure_from_product_name','extracted_units']]\
.head()




In [ ]:


df_sales[(df_sales['oz_measure_from_product_name'].notna())]




# 4. coverage

In [ ]:
df_sales

In [ ]:
##
df_coverage = pd.DataFrame()
for p in df_sales[df_sales['tx_unit_of_measure'] != 'Grams']['tx_product_category_name'].unique():
    print(p)
    df_temp = pd.DataFrame([mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)&(mba.company_sales_df['extracted_units'].notna())].shape[0]\
    / mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)].shape[0]])
    df_temp['category'] = p
    df_coverage = df_coverage.append(df_temp)




# 5. sanity checks

In [ ]:
df_sales[df_sales['tx_product_category_name'] == 'Flower (packaged quarter - each)']

In [ ]:
###
#mba.company_costs_df.groupby(['product_category_name','shipped_unit_of_measure']).count()

In [ ]:
###
#mba.company_sales_df[mba.company_sales_df['tx_product_category_name'] == 'Concentrate']

In [ ]:
pd.DataFrame([mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Flower (packaged quarter - each)')&(mba.company_sales_df['extracted_units'].notna())].shape[0]\
/ mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Flower (packaged quarter - each)')].shape[0]])





In [ ]:

mba.company_sales_df[mba.company_sales_df['tx_product_category_name'] == 'Flower (packaged quarter - each)'].extracted_units.value_counts(normalize = False)

In [ ]:
mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Capsule (weight - each)')]



In [ ]:
##
df = pd.DataFrame()
for p in mba.company_sales_df['tx_product_category_name'].unique():
    print(p)
    df_temp = mba.company_sales_df[mba.company_sales_df['tx_product_category_name'] == p].extracted_units.value_counts(normalize = True).to_frame().sort_values(by ='extracted_units',ascending = False )
    df_temp['category'] = p
    df = df.append(df_temp)




In [ ]:
##
df_coverage = pd.DataFrame()
for p in mba.company_sales_df['tx_product_category_name'].unique():
    print(p)
    df_temp = pd.DataFrame([mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)&(mba.company_sales_df['extracted_units'].notna())].shape[0]\
    / mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == p)].shape[0]])
    df_temp['category'] = p
    df_coverage = df_coverage.append(df_temp)




In [ ]:
df_coverage

In [ ]:
#df.to_csv('dominant_unit.csv')

In [ ]:
mba.company_sales_df[(mba.company_sales_df['tx_product_category_name'] == 'Concentrate (Each)')&(mba.company_sales_df['extracted_units'].isna())]



In [ ]:
###
mba.company_sales_df[(mba.company_sales_df.tx_product_category_name == 'Flower (packaged quarter - each)')&(mba.company_sales_df['extracted_units'].notna())][['adjusted_tx_price_per_unit','sales_month']].groupby(['sales_month']).median()



In [ ]:
mba.company_sales_df.adjusted_tx_price_per_unit.isna().sum()

In [ ]:
mba.company_sales_df.adjusted_tx_price_per_unit.shape

In [ ]:
temp = mba.company_sales_df.copy()

In [ ]:
(temp.letter_gram_measure_from_product_name.isna() == False).sum()

In [ ]:
temp.head()

In [ ]:
((temp.tx_unit_of_measure == 'Each') & (temp.extracted_units.isna() == True) & (temp.measurement_ratio_vs_gram == 1)).sum()

In [ ]:
## Coverage %
1 - (299024/1405912)

In [ ]:
mba.run_analysis('tx_product_category_name', 'Pre-Roll Flower', 'S')

In [ ]:
mba.run_analysis('product_category_name', 'Pre-Roll Flower', 'C')

In [ ]:
mba.run_analysis('combined_product_category', 'flower', 'S')

In [ ]:
mba.run_analysis('combined_product_category', 'flower', 'C')

In [ ]:
###
#mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged quarter - each)'][['adjusted_tx_price_per_unit','sales_month']].groupby(['sales_month']).mean()



In [ ]:
combined_median_price = mba.company_sales_df[mba.company_sales_df.combined_product_category == 'flower'].adjusted_tx_price_per_unit.median()

eighth_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged eighth - each)'].tx_price_per_unit.median()
quarter_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged quarter - each)'].tx_price_per_unit.median()
half_once_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged half ounce - each)'].tx_price_per_unit.median()
once_median_price = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower (packaged ounce - each)'].tx_price_per_unit.median()

In [ ]:
combined_median_price_flower_non_labeled = mba.company_sales_df[mba.company_sales_df.tx_product_category_name == 'Flower'].adjusted_tx_price_per_unit.median()


In [ ]:
combined_median_price

In [ ]:
combined_median_price_flower_non_labeled

In [ ]:
eighth_median_price / 3.5

In [ ]:
quarter_median_price / 7

In [ ]:
half_once_median_price / 14

In [ ]:
once_median_price / 28

In [ ]:
mba.company_sales_df[mba.company_sales_df.tx_product_category_name.str.contains('Flower', case = False)][['tx_price_per_unit', 'adjusted_tx_price_per_unit', 'tx_product_category_name']].groupby('tx_product_category_name').describe()

We see that adjusted_price is overall on a smaller scale than tx_price_per_unit due to units being converted down to grams. 

In [ ]:
mba.output_time_series_metadata()

In [ ]:
mba.save()

In [ ]:
import pickle
picklefile = open('msrp_band_analyzer_training_object', 'rb')
old_mba = pickle.load(picklefile)

In [ ]:
old_mba['company_sales_df'].head()

In [ ]:
new_mba = msrp_band_analyzer.MSRPBand(company_costs_df = old_mba['company_costs_df'], company_sales_df = old_mba['company_sales_df'])

In [ ]:
new_mba.company_sales_df.head()

In [ ]:
mba.msrp_summary_table_by_time

# edible

In [ ]:
df_sales.head()

In [ ]:
#df_sales[df_sales['count_measure_from_product_name'].notna()]['tx_product_category_name'].unique()

In [ ]:
# for p in df_sales['tx_product_category_name'].unique():
#     print(p)
#     if p in ['Shake/Trim (by strain)','Shake/Trim','Infused Butter/Oil (weight - each)','Infused Butter/Oil (volume - each)',\
#             'Shake/Trim allocated for extraction']:
#         continue
#     else:
#         temp = df_sales[df_sales['tx_product_category_name'] == p]
#         temp_gb = temp[['letter_gram_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name']).count().\
#         reset_index()['letter_gram_measure_from_product_name'].str.lower()
#         #print(temp_gb)
#         temp_gb.to_csv(str(p) + '.csv')

In [249]:
data= df_sales[df_sales['tx_product_category_name'] == 'Pre-Roll Flower']




In [250]:
data.shape[0]

143547

In [251]:
#letter gram
data\
[['letter_gram_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name']).count().reset_index().sort_values(by = 'tx_price_per_unit',ascending = False)



,letter_gram_measure_from_product_name,tx_price_per_unit
15,0.5g,30541
37,1g,26721
28,1.0g,9321
5,.5g,8905
21,0.8g,7395
16,0.6g,5282
22,1 G,5058
50,3.5g,3789
36,1G,2723
23,1 g,1994


In [252]:
#oz
data\
[['oz_measure_from_product_name','tx_price_per_unit']].groupby(['oz_measure_from_product_name']).count().reset_index()



,oz_measure_from_product_name,tx_price_per_unit
0,.09oz,72


In [253]:
# count/piece
data\
[['count_measure_from_product_name','tx_price_per_unit']].groupby(['count_measure_from_product_name']).count().reset_index()



,count_measure_from_product_name,tx_price_per_unit
0,2.0,360
1,3.0,140
2,4.0,1793
3,5.0,1584
4,6.0,2154
5,7.0,417
6,10.0,1857
7,12.0,1334
8,14.0,34
9,15.0,15


In [254]:
data\
[['letter_gram_measure_from_product_name','count_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name','count_measure_from_product_name']).count().reset_index()



,letter_gram_measure_from_product_name,count_measure_from_product_name,tx_price_per_unit
0,.5g,2.0,124
1,.5g,6.0,1593
2,0.375g,12.0,345
3,0.375g,32.0,58
4,0.58g,10.0,38
5,0.5G,7.0,92
6,0.5g,2.0,236
7,0.5g,5.0,1
8,0.5g,7.0,62
9,0.5g,10.0,1183


In [225]:
data\
[['letter_gram_measure_from_product_name','oz_measure_from_product_name','tx_price_per_unit']].groupby(['letter_gram_measure_from_product_name','oz_measure_from_product_name']).count().reset_index()



,letter_gram_measure_from_product_name,oz_measure_from_product_name,tx_price_per_unit
0,2.5g,.09oz,72


In [226]:
data\
[['count_measure_from_product_name','oz_measure_from_product_name','tx_price_per_unit']].groupby(['count_measure_from_product_name','oz_measure_from_product_name']).count().reset_index()



,count_measure_from_product_name,oz_measure_from_product_name,tx_price_per_unit


In [227]:
data[data['oz_measure_from_product_name'].notna()]['tx_product_name'].unique()




array(['Pre-Roll Variety Pack 140/211/343/429/514 2.5g (.09oz)',
       'Pre-Roll Variety Pack 134/232/345/423/515 2.5g (.09oz)'],
      dtype=object)

In [235]:
data[data['count_measure_from_product_name'] == 28].loc[2079071]['tx_product_name']




' Glass House Farms Pre-roll 1.0g Hybrid GG#4 (28ct)'

In [258]:
data[data['letter_gram_measure_from_product_name'] == '1g']



,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,tx_price_per_unit,date_in_month,bad_numbers_from_product_name,extracted_units,letter_gram_measure_from_product_name,count_measure_from_product_name,gram_measure_from_product_name,oz_measure_from_product_name,adjusted_tx_price_per_unit,measurement_ratio_vs_gram,combined_product_category,brand_breakable_by_dash_boolean,brands,brands_by_category
2334225,C9-0000323-LIC,0007790601,active,Consumer,2020-01-24 14:50:55+00:00,2020-01,3,56.65,active,1904688,1A4060300006D0A000050041,Space Coyote Classic Sativa Hash Joint 1g,Pre-Roll Flower,Each,1.0,17.43,17.430,2020-01,10953,1.0,1g,NaN,NaN,NaN,17.430,1.0,pre-roll flower,False,Space Coyote Classic Sativa Hash Joint 1g,Space Coyote Classic Sativa Hash Joint 1g
2340111,C9-0000323-LIC,0007033077,active,Consumer,2020-01-18 15:30:57+00:00,2020-01,3,95.86,active,1905105,1A4060300006D0A000050058,Space Coyote Classic Hybrid Hash Joint 1g (24ct),Pre-Roll Flower,Each,2.0,34.86,17.430,2020-01,11950,1.0,1g,24.0,NaN,NaN,17.430,1.0,pre-roll flower,False,Space Coyote Classic Hybrid Hash Joint 1g (24ct),Space Coyote Classic Hybrid Hash Joint 1g (24ct)
2339804,C9-0000323-LIC,0007052859,active,Consumer,2020-01-18 17:50:54+00:00,2020-01,2,41.83,active,1905116,1A4060300006D0A000050069,Space Coyote Classic Indica Hash Joint 1g,Pre-Roll Flower,Each,2.0,34.86,17.430,2020-01,10952,1.0,1g,NaN,NaN,NaN,17.430,1.0,pre-roll flower,False,Space Coyote Classic Indica Hash Joint 1g,Space Coyote Classic Indica Hash Joint 1g
2338601,C9-0000323-LIC,0007227785,active,Consumer,2020-01-20 01:11:12+00:00,2020-01,2,33.99,active,1905116,1A4060300006D0A000050069,Space Coyote Classic Indica Hash Joint 1g,Pre-Roll Flower,Each,1.0,17.43,17.430,2020-01,10952,1.0,1g,NaN,NaN,NaN,17.430,1.0,pre-roll flower,False,Space Coyote Classic Indica Hash Joint 1g,Space Coyote Classic Indica Hash Joint 1g
2326468,C9-0000323-LIC,0008763828,active,Consumer,2020-01-31 20:58:12+00:00,2020-01,4,73.20,active,1904688,1A4060300006D0A000050041,Space Coyote Classic Sativa Hash Joint 1g,Pre-Roll Flower,Each,1.0,17.43,17.430,2020-01,10953,1.0,1g,NaN,NaN,NaN,17.430,1.0,pre-roll flower,False,Space Coyote Classic Sativa Hash Joint 1g,Space Coyote Classic Sativa Hash Joint 1g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37997,C10-0000695-LIC,0177587274,active,Consumer,2022-04-04 16:13:13+00:00,2022-04,2,23.10,active,22274668,1A4060300003BC9000081867,1LYFE - Sugar Hill Preroll (1g),Pre-Roll Flower,Each,1.0,11.55,11.550,2022-04,NaN,1.0,1g,NaN,NaN,NaN,11.550,1.0,pre-roll flower,True,1LYFE,1LYFE (Pre-Roll Flower)
38473,C10-0000408-LIC,0177561347,active,Consumer,2022-04-04 14:36:24.903000+00:00,2022-04,1,22.93,active,22906180,1A4060300006027000149270,Around The World – 1g CUREjoint 3 Pack - Forei...,Pre-Roll Flower,Each,1.0,22.93,22.930,2022-04,NaN,1.0,1g,NaN,NaN,NaN,22.930,1.0,pre-roll flower,True,Around The World – 1g CUREjoint 3 Pack,Around The World – 1g CUREjoint 3 Pack (Pre-Ro...
38099,C10-0000169-LIC,0177582350,active,Consumer,2022-04-04 15:53:50.950000+00:00,2022-04,2,39.18,active,23028288,1A4060300006027000150091,Rainbow Sherbet - 1g CUREjoint Single,Pre-Roll Flower,Each,2.0,19.59,9.795,2022-04,NaN,1.0,1g,NaN,NaN,NaN,9.795,1.0,pre-roll flower,True,Rainbow Sherbet,Rainbow Sherbet (Pre-Roll Flower)
38100,C10-0000169-LIC,0177582350,active,Consumer,2022-04-04 15:53:50.950000+00:00,2022-04,2,39.18,active,23028281,1A4060300006027000150086,Bacio Gelato - 1g CUREjoint Single,Pre-Roll Flower,Each,2.0,19.59,9.795,2022-04,NaN,1.0,1g,NaN,NaN,NaN,9.795,1.0,pre-roll flower,True,Bacio Gelato,Bacio Gelato (Pre-Roll Flower)


In [256]:
data[(data['letter_gram_measure_from_product_name'] == '0.5g')]['tx_price_per_unit'].describe()



count    30541.000000
mean        23.712199
std         15.094777
min          0.030000
25%          8.760000
50%         21.690000
75%         35.020000
max         66.660000
Name: tx_price_per_unit, dtype: float64

In [257]:
data[(data['letter_gram_measure_from_product_name'] == '1g')]['tx_price_per_unit'].describe()



count    26721.000000
mean         9.593140
std          7.764475
min          0.030000
25%          5.225000
50%          8.510000
75%         11.320000
max         74.250000
Name: tx_price_per_unit, dtype: float64

In [ ]:
data[data['tx_product_name'] == ' Raw Garden Cartridge 1.0g Indica Fire Walker (10ct)']



In [ ]:
df_sales[df_sales['tx_product_category_name'] == 'Infused (edible)']\
[['count_measure_from_product_name','letter_gram_measure_from_product_name','tx_price_per_unit']].groupby(['count_measure_from_product_name','letter_gram_measure_from_product_name']).count().reset_index()




In [ ]:
edible = df_sales[df_sales['tx_product_category_name'] == 'Shake (Packaged Eighth - each)']






In [ ]:
edible[edible['letter_gram_measure_from_product_name'].notna()]['tx_product_name'].unique()

In [ ]:
pd.DataFrame(edible[edible['letter_gram_measure_from_product_name'].notna()]['tx_product_name'].unique())







In [ ]:
edible[['oz_measure_from_product_name','tx_price_per_unit']].groupby(['oz_measure_from_product_name']).count()




# Testing

In [ ]:
from underwriting import msrp_band_analyzer_testing


In [ ]:
result = msrp_band_analyzer_testing.run_test_set_analysis('product_category_name', 'Flower', 'C')

In [ ]:
result[0]

In [ ]:
result[1]

In [ ]:
result = run_test_set_analysis('tx_product_category_name', 'Pre-Roll Flower', 'S')